In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Catboost
from catboost import CatBoostRegressor

#Sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

#SkForecast
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.preprocessing import TimeSeriesDifferentiator

In [7]:
df= pd.read_csv("/Users/santi/Documents/csv/datos_arreglados.csv",sep=",",index_col=0)

df["date"]=pd.to_datetime(df["date"])
df.set_index("date", inplace=True)

df.head(5)

,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales
date,,,,,,,,,,,
2013-07-08,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,0,0,12.7414,0.0000
2013-07-15,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,28,2013,1,0,12.7414,0.0000
2013-07-22,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,29,2013,0,4,10.9858,43.9432
2013-07-29,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,30,2013,0,6,10.9858,65.9148
2013-08-05,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End(BOS_1),Boston,31,2013,0,0,10.9858,0.0000


In [57]:
df_boston_1= df[df["stores"]== "South_End(BOS_1)"].copy()
df_boston_1.tail()

,item,category,department,stores,region,week,year,is_holidays,units_sold,sell_price,total_sales
date,,,,,,,,,,,
2016-03-28,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,12,2016,1,3,1.2,3.6
2016-04-04,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,13,2016,0,6,1.2,7.2
2016-04-11,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,14,2016,0,17,1.2,20.4
2016-04-18,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,15,2016,0,20,1.2,24.0
2016-04-25,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,South_End(BOS_1),Boston,16,2016,0,5,1.2,6.0


In [66]:
df_boston_1fit = pd.pivot_table(
           data    = df_boston_1,
           values  = 'total_sales',
           index   = 'date',
           columns = 'item'
       )

#df_boston_1fit = df_boston_1fit.asfreq('1W')

In [67]:
df_boston_1fit.fillna(0, inplace=True)

In [68]:
df_boston_1fit.head(50)

item,ACCESORIES_1_001,ACCESORIES_1_002,ACCESORIES_1_003,ACCESORIES_1_004,ACCESORIES_1_005,ACCESORIES_1_006,ACCESORIES_1_007,ACCESORIES_1_008,ACCESORIES_1_009,ACCESORIES_1_010,...,SUPERMARKET_3_818,SUPERMARKET_3_819,SUPERMARKET_3_820,SUPERMARKET_3_821,SUPERMARKET_3_822,SUPERMARKET_3_823,SUPERMARKET_3_824,SUPERMARKET_3_825,SUPERMARKET_3_826,SUPERMARKET_3_827
date,,,,,,,,,,,,,,,,,,,,,
2011-01-31,0.0,0.0000,0.0,5.7722,0.0000,0.0,0.0,1.2236,0.0000,0.0000,...,0.0,0.000,3.768,0.0,9.552,6.000,2.964,0.000,0.0,0.0
2011-02-07,0.0,0.0000,0.0,75.0386,0.0000,0.0,0.0,18.3540,12.4488,8.4322,...,0.0,0.000,16.956,0.0,66.864,24.000,2.964,57.600,0.0,0.0
2011-02-14,0.0,0.0000,0.0,46.1776,0.0000,0.0,0.0,12.8478,37.3464,4.2161,...,0.0,0.000,16.956,0.0,14.328,21.000,5.928,43.200,0.0,0.0
2011-02-21,0.0,0.0000,0.0,34.6332,0.0000,0.0,0.0,16.7580,18.6732,16.8644,...,0.0,23.760,9.420,0.0,0.000,114.000,14.820,67.200,0.0,0.0
2011-02-28,0.0,0.0000,0.0,92.3552,0.0000,0.0,0.0,11.1720,2.0748,16.8644,...,0.0,0.000,11.304,0.0,0.000,93.000,8.892,14.400,0.0,0.0
2011-03-07,0.0,0.0000,0.0,46.1776,0.0000,0.0,0.0,3.9102,29.0472,12.6483,...,0.0,0.000,18.840,0.0,0.000,75.000,11.904,33.600,0.0,0.0
2011-03-14,0.0,21.1204,0.0,28.8610,0.0000,0.0,0.0,24.5784,8.2992,29.5127,...,0.0,0.000,15.072,0.0,0.000,57.000,11.904,19.200,0.0,0.0
2011-03-21,0.0,5.2801,0.0,69.2664,0.0000,0.0,0.0,14.5236,32.5185,15.8004,...,0.0,0.000,9.420,0.0,0.000,60.000,8.928,14.400,0.0,0.0
2011-03-28,0.0,0.0000,0.0,69.2664,0.0000,0.0,0.0,15.0822,2.1679,15.8004,...,0.0,0.000,9.420,0.0,0.000,45.000,14.280,9.600,0.0,0.0


In [69]:
pipeline = make_pipeline(TimeSeriesDifferentiator(order=1),StandardScaler())
pipeline

Pipeline(steps=[('timeseriesdifferentiator', TimeSeriesDifferentiator()),
                ('standardscaler', StandardScaler())])

In [70]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = CatBoostRegressor(random_state=123),
                 lags               = 12,
                 transformer_exog   = pipeline,
                 weight_func        = None,
                 series_weights     = None
             )

In [ ]:
forecaster.fit(df_boston_1fit)

In [ ]:
forecaster.predict(steps=8)

/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/site-packages/skforecast/utils/utils.py:867: UserWarning: `last_window` has DatetimeIndex index but no frequency. Index is overwritten with a RangeIndex of step 1.
  warnings.warn(
/Users/santi/anaconda3/envs/Forecasting/lib/python3.11/s

,ACCESORIES_1_001,ACCESORIES_1_002,ACCESORIES_1_003,ACCESORIES_1_004,ACCESORIES_1_005,ACCESORIES_1_006,ACCESORIES_1_007,ACCESORIES_1_008,ACCESORIES_1_009,ACCESORIES_1_010,...,SUPERMARKET_3_818,SUPERMARKET_3_819,SUPERMARKET_3_820,SUPERMARKET_3_821,SUPERMARKET_3_822,SUPERMARKET_3_823,SUPERMARKET_3_824,SUPERMARKET_3_825,SUPERMARKET_3_826,SUPERMARKET_3_827
12,27.789833,4.140007,5.340392,35.404183,24.259574,3.667485,19.706989,7.735716,7.543002,11.475853,...,44.807741,5.106871,24.403246,16.391657,46.673715,4.381045,3.406369,13.515423,6.451911,11.626431
13,27.999971,4.499690,6.197417,34.228707,24.789798,4.257673,20.330698,7.559363,9.273692,10.816602,...,43.910486,5.128068,27.211202,14.564747,43.747791,5.242168,3.865134,14.350251,6.521715,11.164204
14,26.328921,4.788007,5.262286,33.524074,23.008549,4.640423,22.710498,8.071185,8.621348,11.218715,...,41.835392,4.648711,27.383775,15.549981,44.153487,4.791660,3.686927,14.872233,7.055823,11.734744
15,27.510873,4.666759,5.014225,37.225253,24.579298,4.555125,23.435233,7.901096,8.564310,10.172220,...,43.451139,5.081482,25.899328,16.163581,45.184560,5.005135,4.157513,12.873321,7.700567,11.445270
16,25.709823,4.794602,5.873801,36.425272,23.581916,4.554126,23.030048,8.205399,8.881884,10.892707,...,45.941179,4.900526,24.182016,14.631621,42.229588,4.839088,4.439601,13.413846,7.618448,11.110878
17,25.499767,4.680408,5.003648,37.421172,23.281286,4.397286,23.424297,8.376600,9.305838,11.198156,...,41.663397,4.956112,24.407144,13.970194,42.520634,4.765676,4.470983,12.716390,7.824679,10.505437
18,25.090734,4.809203,5.211955,36.594984,23.419352,4.467930,22.438403,8.218596,8.976753,10.794658,...,37.464112,5.047916,25.061311,14.790585,41.167197,4.900523,4.533869,13.147680,7.632792,10.742144
19,26.474191,4.847750,5.130715,39.839137,23.870233,4.589681,26.174269,8.902757,9.223312,10.077641,...,40.663032,4.943305,25.472313,13.207084,42.725239,4.802711,4.340104,12.867508,7.623198,11.087638
